<a href="https://colab.research.google.com/github/shivanisingh73/04-03-19-Ajax/blob/master/hw3_AI_Ada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import pandas as pd
from pprint import pprint
import math
import random


def check_wordlength(line):
  result = False
  for word in line:
    if len(word) > 12:
      return False
  return True


def check_article(line):
  articles = [ "de", "het", "een", "der", "des", "den" ]
  for word in line:
    if word in articles:
      return False
  return True


def check_prepositions(line):
  prepositions = {
	"à", "aan", "aangaande", "achter", "behalve", "behoudens", "beneden", "benevens", "benoorden", "benoordoosten", "benoordwesten",
	"beoosten", "betreffende", "bewesten", "bezijden", "bezuiden", "bezuidoosten", "bezuidwesten", "bij", "binnen", "blijkens", "boven", "bovenaan",
	"buiten", "circa", "conform", "contra", "cum", "dankzij", "door", "gedurende", "gezien", "in", "ingevolge", "inzake", "jegens", "krachtens",
	"langs", "luidens", "met", "middels", "na", "naar", "naast", "nabij", "namens", "nevens", "niettegenstaande", "nopens", "om",
	"omstreeks", "omtrent", "onder", "onderaan", "ongeacht", "onverminderd", "op", "over", "overeenkomstig", "per", "plus", "post",
	"richting", "rond", "rondom", "spijts", "staande", "te", "tegen", "tegenover", "ten", "ter", "tijdens", "tot", "tussen",
	"uit", "van", "vanaf", "vanuit", "versus", "via", "vis-à-vis", "volgens", "voor", "voorbij", "wegens", "zijdens",
	"zonder",
}
  for word in line:
    if word in prepositions:
      return False
  return True
 

def check_coordinatingConjunctions(line):
  conjunctions = [ "and", "or", "and/or", "yet", "sooner", "just", "only", "if", "even" ]
  for word in line:
    if word in conjunctions:
      return True
  return False
 


def check_pronouns(line):
  pronouns = [ "ik", "je", "jij", "hij", "ze", "we", "wij", "jullie", "zij", "u", "ge", "gij",
              "men", "mij", "jou", "hem", "haar", "hen", "hun", "uw", "dit", "dat", "deze", "die",
              "zelf", "mijn", "mijne", "jouw", "jouwe", "zijne", "hare", "ons", "onze", "hunne", "uwe",
              "elkaars", "elkanders", "alle", "sommige", "sommigen", "weinig", "weinige", "weinigen",
              "veel", "vele", "velen", "geen", "beetje", "elke", "elk", "genoeg", "meer", "meest",
              "meeste", "meesten", "paar", "zoveel", "enkele", "enkelen", "zoveelste", "hoeveelste",
	            "laatste", "laatsten", "iedere", "allemaal", "zekere", "ander", "andere", "gene",
              "enig", "enige", "verscheidene", "verschillende", "voldoende", "allerlei",
              "allerhande", "enerlei", "enerhande", "beiderlei", "beiderhande", "tweeërlei", "tweeërhande",
	            "drieërlei", "drieërhande", "velerlei", "velerhande", "menigerlei", "menigerhande", "enigerlei",
              "enigerhande", "generlei", "generhande", "mezelf", "mijzelf", "jezelf", "jouzelf", "zichzelf", "haarzelf", "hemzelf", "onszelf", "julliezelf",
	            "henzelf", "hunzelf", "uzelf", "zich", "mekaar", "elkaar", "elkander", "mekander", "iedereen",
              "ieder", "eenieder", "alleman", "allen", "alles", "iemand", "niemand", "iets", "niets", "menigeen" ]
  for word in line:
    if word in pronouns:
      return False
  return True

 
def apply_feature_train(filename):
  data = []
  with open(filename,'r') as r:
      line = r.readline()
      while line != '':
        each_line = []
        lang = False
        if line[:2] == 'en':
          lang = True
        row = line[3:]
        row = row.strip()
        row = row.split(' ')
        each_line.append(check_wordlength(row))
        each_line.append(check_article(row))
        each_line.append(check_prepositions(row))
        each_line.append(check_coordinatingConjunctions(row))
        each_line.append(check_pronouns(row))
        each_line.append(lang)
        data.append(each_line)
        # print(line, end='')
        line = r.readline()
  # print("\n"+str(data))
  # return data
  with open('input_dt.dat','w') as f1:
    for each in data:
      for item in each:
        f1.write(str(item)+' ')
      f1.write('\n')


def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    uniqueVal, counts1 = np.unique(column, return_counts = True)
    # Divide by the total column length to get a probability
    probabilities1 = []
    for each in counts1:
      probabilities1.append(each / len(column))
    probabilities1 = counts1 / len(column)

    # Initialize the entropy to 0
    entropy1 = 0
    
    entropy1 = np.sum([prob * np.log2(prob) for prob in probabilities1]) 

    return -entropy1


def calc_feature_entropy(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    vals,counts = np.unique(data[split_name], return_counts=True)

    subsets = []

    for i in range (len(vals)):
      split = data[data[split_name] == vals[i]]
      split.dropna()
      subsets.append(calc_entropy(split[target_name]))

    to_subtract1 = np.sum([(counts[i]/np.sum(counts))*subsets[i] for i in range(len(vals))])

    return to_subtract1


def give_sample_weight(data):
  no_of_samples = len(data.index)
  sample_weight = 1/no_of_samples
  data[6] = [1/no_of_samples] * no_of_samples
  return data, sample_weight


def base_learners(data, features, tree):
  entropy_lst = []

  for f in features:
    entropy_lst.append(calc_feature_entropy(data, f, 5))
    # entropy_lst.append(calc_feature_entropy(data[f]))
  min_entropy = min(entropy_lst)

  split_feature = features[entropy_lst.index(min_entropy)]
  tree = {split_feature:{}}
  values = data[split_feature].unique()

  for i in range(len(values)):
      child_data = data[data[split_feature] == values[i]]
      tree[split_feature][values[i]] = child_data

  return tree, split_feature


def performance(te):
  if te==1:
    return 0
  if te==0:
    return 1
  p = ((1 - te)/ te)
  performance = (1/2) *(math.log(p))
  return performance


def mis_classified(tree, split_feature, target_atr,data):
  # print(data)
  inc_indexes = data.index[data[5] != data[split_feature]].tolist()
  incorrect = 0
  # print(tree[split_feature])
  # print(tree[3][True])
  for flag in tree[split_feature]:
    for i in tree[split_feature][flag][5]:
      if flag!=i:
        incorrect+=1
  return incorrect, inc_indexes


def update_weights(data, inc_indexes, sample_weight, performance):
  data[7] = [1] * len(data.index)
  sum = 0
  for ind in data.index:
    if ind not in inc_indexes:
      data.loc[ind, 7] = sample_weight * math.exp(-performance)
    else:
      data.loc[ind, 7] = sample_weight * math.exp(performance)
    sum += float(data.loc[ind, 7])
  return data, sum


def normalize_weight(data, sum, sample_weight, performance):
  for ind in data.index:
    data.loc[ind, 7] = data.loc[ind, 7]/ sum
  return data


def rangify(data):
  range_data = [[0, data.loc[0, 7]]]
  for i in range(1, len(data)):
    start_num = range_data[i-1][1]
    range_data.append([start_num, start_num+data.loc[i, 7]])
  # print(range_data)
  new_data = []
  for j in range(0, len(data)):
    rnd_no = random.random()
    for i in range(len(range_data)):
      if range_data[i][0]<=rnd_no<=range_data[i][1]:
        new_data.append(list(data.iloc[i]))
        break
  new_df = pd.DataFrame(new_data)
  new_df = new_df.drop(new_df.columns[[6,7]], axis=1)
  return new_df


def adaboost(data,features,ans):
  for _ in range(10):
    tree={}
    data,sample_weight = give_sample_weight(data)
    tree, split_feature = base_learners(data, features, tree)
    ans.append(split_feature)
    inc, inc_indexes = mis_classified(tree, split_feature, 5,data)
    te = inc/ len(data)
    pe = performance(te)
    data, sum= update_weights(data, inc_indexes, sample_weight, pe)
    data = normalize_weight(data, sum, sample_weight, performance) 
    data = rangify(data)


def runFeature(feature_no,line):
  arr = line.strip().split(" ")
  if feature_no == 0:
    return check_wordlength(arr)
  elif feature_no==1:
    return check_article(arr)
  elif feature_no == 2:
    return check_prepositions(arr)
  elif feature_no == 3:
    return check_coordinatingConjunctions(arr)
  elif feature_no == 4:
    return check_pronouns(arr)


def testData(line,ans):
  en,nl=0,0
  for i in ans:
    if runFeature(i,line):
      en+=1
    else:
      nl+=1
  if en>nl:
    print("en")
  else:
    print("nl")



# tree = {}
features = [0, 1, 2, 3, 4]
apply_feature_train('input.dat')
file_data = pd.read_csv('input_dt.dat', sep=" ", header=None)
file_data.drop(file_data.columns[[6]], axis=1, inplace=True)
ans = []
adaboost(file_data,features,ans)
# print(ans)

#line = "als station, terwijl de stationschef in de dienstwoning uit 1839 bleef wonen. Pas in 1931 "
#testData(line,ans)
with open("String.txt") as f:
  for line in f:
    line=line.strip()
    testData(line,ans)

# print(file_data)
# data, sample_weight = give_sample_weight(file_data)
# # print(data)
# tree, split_feature = base_learners(data, features, tree)
# # pprint(tree)
# inc, inc_indexes = mis_classified(tree, split_feature, 5)
# te = inc/ len(data)
# pe = performance(te)
# data, sum= update_weights(data, inc_indexes, sample_weight, pe)
# # print(data)
# data = normalize_weight(data, sum, sample_weight, performance) 
# print(data)
# data = rangify(data)
# print(data)

nl
nl
en
nl
en
nl
nl
nl
nl
nl
